In [1]:
import cvxpy as cp
import numpy as np

# Define currencies and exchange rates
currencies = ['USD', 'EUR', 'GBP', 'JPY', 'CAD']
exchange_rates = {
    ('USD', 'EUR'): 0.900,
    ('USD', 'GBP'): 0.770,
    ('USD', 'JPY'): 140.000,
    ('USD', 'CAD'): 1.350,
    ('EUR', 'USD'): 1.111,
    ('EUR', 'GBP'): 0.860,
    ('EUR', 'JPY'): 155.000,
    ('EUR', 'CAD'): 1.500,
    ('GBP', 'USD'): 1.300,
    ('GBP', 'EUR'): 1.163,
    ('GBP', 'JPY'): 180.000,
    ('GBP', 'CAD'): 1.750,
    ('JPY', 'USD'): 0.007,
    ('JPY', 'EUR'): 0.0064,
    ('JPY', 'GBP'): 0.0056,
    ('JPY', 'CAD'): 0.010,
    ('CAD', 'USD'): 0.741,
    ('CAD', 'EUR'): 0.667,
    ('CAD', 'GBP'): 0.571,
    ('CAD', 'JPY'): 100.000
}

# Convert exchange rates to costs (negative logarithm)
costs = {edge: -np.log(rate) for edge, rate in exchange_rates.items()}

# Define variables
flow = {edge: cp.Variable(boolean=True) for edge in exchange_rates}

# Objective function: minimize the total cost
objective = cp.Minimize(sum(costs[edge] * flow[edge] for edge in exchange_rates))

# Constraints
constraints = []

# Flow conservation constraints (ensure flow forms a cycle)
for currency in currencies:
    inflow = sum(flow[(i, currency)] for i in currencies if (i, currency) in exchange_rates)
    outflow = sum(flow[(currency, j)] for j in currencies if (currency, j) in exchange_rates)
    constraints.append(inflow == outflow)

# Ensure there is at least one cycle (total flow >= 1)
constraints.append(sum(flow[edge] for edge in exchange_rates) >= 1)

# Define and solve the problem
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print the results
if result < 0:
    print("Arbitrage opportunity exists!")
    for edge in exchange_rates:
        if flow[edge].value > 0.5:
            print(f"Use edge {edge} with exchange rate {exchange_rates[edge]}")
else:
    print("No arbitrage opportunity found.")

Arbitrage opportunity exists!
Use edge ('USD', 'EUR') with exchange rate 0.9
Use edge ('USD', 'JPY') with exchange rate 140.0
Use edge ('EUR', 'GBP') with exchange rate 0.86
Use edge ('EUR', 'JPY') with exchange rate 155.0
Use edge ('GBP', 'USD') with exchange rate 1.3
Use edge ('GBP', 'CAD') with exchange rate 1.75
Use edge ('JPY', 'GBP') with exchange rate 0.0056
Use edge ('JPY', 'CAD') with exchange rate 0.01
Use edge ('CAD', 'USD') with exchange rate 0.741
Use edge ('CAD', 'EUR') with exchange rate 0.667
